# Natural Language Processing - Word Embedding A
Authors: [RedSkittleFox](https://github.com/RedSkittleFox), [DJAltair](https://github.com/DJAltair)


## 1.0 Predict the Countries from Capitals



###  1.1 Importing the data



The test dataset for prediction country capitals will be loaded as a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html),


In [45]:
# Run this cell to import packages.
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [46]:
# pulling resources necessary to run code
# they will be stored in '/content/nlp-resources-edu/05_Embedding_lab/' folder

!git clone  https://github.com/mmazurek-wat/nlp-resources-edu.git
resource_path = './nlp-resources-edu/05_Embedding_lab/'

sys.path.append(resource_path)

fatal: destination path 'nlp-resources-edu' already exists and is not an empty directory.


In [47]:
from os import path
capitals_file = 'capitals.txt'
capitals_file_path = path.join(resource_path, capitals_file)
data = pd.read_csv(capitals_file_path, delimiter=' ')
data.columns = ['city1', 'country1', 'city2', 'country2']

# print first five elements in the DataFrame
data.head(5)

,city1,country1,city2,country2
0,Athens,Greece,Bangkok,Thailand
1,Athens,Greece,Beijing,China
2,Athens,Greece,Berlin,Germany
3,Athens,Greece,Bern,Switzerland
4,Athens,Greece,Cairo,Egypt


***

### To Run This Code On Full Google Dataset:


If you want to download the full dataset on your own and choose your own set of word embeddings,
please see the instructions and some helper code.

- Download the dataset from this [page](https://code.google.com/archive/p/word2vec/).
- Search in the page for 'GoogleNews-vectors-negative300.bin.gz' and click the link to download.

In [48]:
import gensim.downloader as api

embeddings = api.load('word2vec-google-news-300')

vec_king = embeddings['king']

In [49]:
import nltk
from gensim.models import KeyedVectors


#embeddings = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary = True)

#embedding as dictionary


word_embeddings = {}
for word in embeddings.key_to_index:
  word_embeddings[word] = embeddings[word]

print(len(word_embeddings))

3000000


***

### To Run On Sample Dataset provided



In [50]:
embedding_subset_file = 'word_embeddings_subset.p'
embedding_subset_file_path = path.join(resource_path, embedding_subset_file)

word_embeddings = pickle.load(open(embedding_subset_file_path, "rb"))
len(word_embeddings)  # there should be 243 words that will be used in this assignment

243

Each of the word embedding is a 300-dimensional vector.

In [51]:
print(* word_embeddings.keys())

country city China Iraq oil town Canada London England Australia Japan Pakistan Iran gas happy Russia Afghanistan France Germany Georgia Baghdad village Spain Italy Beijing Jordan Paris Ireland Turkey Egypt Lebanon Taiwan Tokyo Nigeria Vietnam Moscow Greece Indonesia sad Syria Thailand Libya Zimbabwe Cuba Ottawa Tehran Sudan Kenya Philippines Sweden Poland Ukraine Rome Venezuela Switzerland Berlin Bangladesh Portugal Ghana Athens king Madrid Somalia Dublin Qatar Chile Islamabad Bahrain Nepal Norway Serbia Kabul continent Brussels Belgium Uganda petroleum Cairo Denmark Austria Jamaica Georgetown Bangkok Finland Peru Romania Bulgaria Hungary Vienna Kingston Manila Cyprus Azerbaijan Copenhagen Fiji Tunisia Kazakhstan queen Beirut Jakarta Croatia Belarus Algeria Malta Morocco Rwanda Bahamas Damascus Ecuador Angola Canberra Liberia Honduras Tripoli Slovakia Doha Armenia Taipei Oman Nairobi Santiago Guinea Uruguay Stockholm Slovenia Zambia Havana Uzbekistan Belgrade Mogadishu Khartoum Botswa

In [52]:
print("dimension: {}".format(word_embeddings['Spain'].shape[0]))

dimension: 300


### Predict relationships among words

Now you will write a function that will use the word embeddings to predict relationships among words.
* The function will take as input three words.
* The first two are related to each other.
* It will predict a 4th word which is related to the third word in a similar manner as the two first words are related to each other.
* As an example, "Athens is to Greece as Bangkok is to ______"?
* You will write a program that is capable of finding the fourth word.
* We will give you a hint to show you how to compute this.

**pogrubiony tekst**

You will implement a function that can tell you the capital of a country.
You should use the same methodology shown in the figure above. To do this,
compute you'll first compute cosine similarity metric or the Euclidean distance.

### 1.2 Cosine Similarity

The cosine similarity function is:

$$\cos (\theta)=\frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\|\|\mathbf{B}\|}=\frac{\sum_{i=1}^{n} A_{i} B_{i}}{\sqrt{\sum_{i=1}^{n} A_{i}^{2}} \sqrt{\sum_{i=1}^{n} B_{i}^{2}}}\tag{1}$$

$A$ and $B$ represent the word vectors and $A_i$ or $B_i$ represent index i of that vector.
& Note that if A and B are identical, you will get $cos(\theta) = 1$.
* Otherwise, if they are the total opposite, meaning, $A= -B$, then you would get $cos(\theta) = -1$.
* If you get $cos(\theta) =0$, that means that they are orthogonal (or perpendicular).
* Numbers between 0 and 1 indicate a similarity score.
* Numbers between -1-0 indicate a dissimilarity score.

**Instructions**: Implement a function that takes in two word vectors and computes the cosine distance.

In [53]:
# use numpy.linalg.norm , numpy.dot


def cosine_similarity(A, B):
    '''
    Input:
        A: a numpy array which corresponds to a word vector
        B: A numpy array which corresponds to a word vector
    Output:
        cos: numerical number representing the cosine similarity between A and B.
    '''

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    dot = np.dot(A, B)
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot / (norma * normb)

    ### END CODE HERE ###
    return cos

In [54]:
# feel free to try different words
king = word_embeddings['king']
queen = word_embeddings['queen']

cosine_similarity(king, queen)

0.6510956

**Expected Output**:

$\approx$ 0.6510956

### 1.3 Euclidean distance

You will now implement a function that computes the similarity between two vectors using the Euclidean distance.
Euclidean distance is defined as:

$$ \begin{aligned} d(\mathbf{A}, \mathbf{B})=d(\mathbf{B}, \mathbf{A}) &=\sqrt{\left(A_{1}-B_{1}\right)^{2}+\left(A_{2}-B_{2}\right)^{2}+\cdots+\left(A_{n}-B_{n}\right)^{2}} \\ &=\sqrt{\sum_{i=1}^{n}\left(A_{i}-B_{i}\right)^{2}} \end{aligned}$$

* $n$ is the number of elements in the vector
* $A$ and $B$ are the corresponding word vectors.
* The more similar the words, the more likely the Euclidean distance will be close to 0.

**Instructions**: Write a function that computes the Euclidean distance between two vectors.

In [55]:
# use numpy.linalg.norm

def euclidean(A, B):
    """
    Input:
        A: a numpy array which corresponds to a word vector
        B: A numpy array which corresponds to a word vector
    Output:
        d: numerical number representing the Euclidean distance between A and B.
    """

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # euclidean distance

    d = np.linalg.norm(A - B)

    ### END CODE HERE ###

    return d


In [56]:
# Test your function
euclidean(king, queen)

2.4796925

**Expected Output:**

2.4796925

### 1.4 Finding the country of each capital

Now, you  will use the previous functions to compute similarities between vectors,
and use these to find the capital cities of countries. You will write a function that
takes in three words, and the embeddings dictionary. Your task is to find the
capital cities. For example, given the following words:

- 1: Athens 2: Greece 3: Baghdad,

your task is to predict the country 4: Iraq.

**Instructions**:

1. To predict the capital you might want to look at the *King - Man + Woman = Queen* example above, and implement that scheme into a mathematical function, using the word embeddings and a similarity function.

2. Iterate over the embeddings dictionary and compute the cosine similarity score between your vector and the current word embedding.

3. You should add a check to make sure that the word you return is not any of the words that you fed into your function. Return the one with the highest score.

In [57]:
def get_country(city1, country1, city2, embeddings):
    """
    Input:
        city1: a string (the capital city of country1)
        country1: a string (the country of capital1)
        city2: a string (the capital city of country2)
        embeddings: a dictionary where the keys are words and values are their embeddings
    Output:
        countries: a dictionary with the most likely country and its similarity score
    """
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # store the city1, country 1, and city 2 in a set called group
    group = set((city1, country1, city2))

    # get embeddings of city 1
    city1_emb = embeddings[city1]

    # get embedding of country 1
    country1_emb = embeddings[country1]

    # get embedding of city 2
    city2_emb = embeddings[city2]

    # get embedding of country 2 (it's a combination of the embeddings of country 1, city 1 and city 2)
    # Remember: King - Man + Woman = Queen
    vec = country1_emb - city1_emb + city2_emb

    # Initialize the similarity to -1 (it will be replaced by a similarities that are closer to +1)
    similarity = -1

    # initialize country to an empty string (you will return it as  a tuple)
    country = ''

    # loop through all words in the embeddings dictionary
    for word in embeddings.keys():

        # first check that the word is not already in the 'group' (it is not a part of input dataset)
        if word not in group:

            # get the word embedding
            word_emb = embeddings[word]

            # calculate cosine similarity between embedding of country 2 and the word in the embeddings dictionary
            cur_similarity = cosine_similarity(vec, word_emb)

            # if the cosine similarity is more similar than the previously best similarity...
            if cur_similarity > similarity:

                # update the similarity to the new, better similarity
                similarity = cur_similarity

                # store the country as a tuple, which contains the word and the similarity
                country = (word, similarity)

    ### END CODE HERE ###

    return country

In [58]:
# Testing your function, note to make it more robust you can return the 5 most similar words.
get_country('Athens', 'Greece', 'Cairo', word_embeddings)

('Egypt', 0.7626822)

**Expected Output:**

('Egypt', 0.7626821)

### 1.5 Model Accuracy

Now you will test your new function on the dataset and check the accuracy of the model:

$$\text{Accuracy}=\frac{\text{Correct \# of predictions}}{\text{Total \# of predictions}}$$

**Instructions**: Write a program that can compute the accuracy on the dataset provided for you. You have to iterate over every row to get the corresponding words and feed them into you `get_country` function above.

In [59]:
data

,city1,country1,city2,country2
0,Athens,Greece,Bangkok,Thailand
1,Athens,Greece,Beijing,China
2,Athens,Greece,Berlin,Germany
3,Athens,Greece,Bern,Switzerland
4,Athens,Greece,Cairo,Egypt
...,...,...,...,...
4946,Zagreb,Croatia,Dakar,Senegal
4947,Zagreb,Croatia,Damascus,Syria
4948,Zagreb,Croatia,Dhaka,Bangladesh
4949,Zagreb,Croatia,Doha,Qatar


In [60]:
def get_accuracy(word_embeddings, data):
    '''
    Input:
        word_embeddings: a dictionary where the key is a word and the value is its embedding
        data: a pandas dataframe containing all the country and capital city pairs

    Output:
        accuracy: the accuracy of the model
    '''

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # initialize num correct to zero
    num_correct = 0

    # loop through the rows of the dataframe
    for i, row in data.iterrows():

        # get city1
        city1 = row['city1']

        # get country1
        country1 = row['country1']

        # get city2
        city2 =  row['city2']

        # get country2
        country2 = row['country2']

        # use get_country to find the predicted country2
        predicted_country2, _ = get_country(city1, country1, city2, word_embeddings)

        # if the predicted country2 is the same as the actual country2...
        if country2 == predicted_country2:
            # increment the number of correct by 1
            num_correct += 1

    # get the number of rows in the data dataframe (length of dataframe)
    m = data.shape[0]

    # calculate the accuracy by dividing the number correct by m
    accuracy = num_correct / m

    ### END CODE HERE ###
    return accuracy


In [61]:
accuracy = get_accuracy(word_embeddings, data)
print(f"Accuracy is {accuracy:.2f}")

Accuracy is 0.92


**Expected Output:**

$\approx$ 0.92


## 2.0 Get the analogy based on Euclidean distance


Implement function to find nearest words, which is  closest to requested vector $v$.  (Euclidean distance)


In [62]:
def find_closest_word(v, embeddings, words):
    min_distance = float('inf')
    closest_word = None

    for word in embeddings.key_to_index:
        if word not in words:
            embedding = embeddings[word]
            distance = np.linalg.norm(v - embedding)

            if distance < min_distance:
                min_distance = distance
                closest_word = word

    return closest_word

Implement function to find analogy like *King - Man + Woman = Queen*  for any word combination

In [ ]:
def get_analogy(rel_src1, rel_trg1, rel_src2, embeddings):
    vec_trg2 = embeddings[rel_trg1] - embeddings[rel_src1] + embeddings[rel_src2]
    words = [rel_src1, rel_trg1, rel_src2]
    closest_word = find_closest_word(vec_trg2, embeddings, words)
    return closest_word

*Test* your functions on  your own subset of words and analogies

In [66]:
print(get_analogy('fast', 'slow', 'clever', embeddings))
print(get_analogy('woman', 'queen', 'man', embeddings))
print(get_analogy('oil', 'money', 'woman', embeddings))

clumsy
king
girl
